In [22]:
from API_KEYS import LAST_FM_API_KEY, LAST_FM_USER_AGENT
import requests
import time

In [23]:
url = 'http://ws.audioscrobbler.com/2.0/'
headers = {
    'user-agent': LAST_FM_USER_AGENT
}
payload = {
    'api_key': LAST_FM_API_KEY,
    'method': 'chart.getTopArtists',
    'format': 'json'
}

payload_tag = {
    'tag': 'polish hip hop',
    'api_key': LAST_FM_API_KEY,
    'method': 'tag.getTopArtists',
    'format': 'json',

}
r = requests.get(url=url, headers=headers, params=payload)
r.status_code

200

In [24]:
r_tag = requests.get(url=url, headers=headers, params=payload_tag)

In [30]:
def get_tag_artists(tag, API_KEY=LAST_FM_API_KEY):
    all_artists = []
    page = 1
    max_pages = 20
    params = {
        'tag': tag,
        'api_key': API_KEY,
        'method': 'tag.getTopArtists',
        'format': 'json',
        'page': page,
    }
    while True:

        response = requests.get(url=url, headers=headers, params=params)
        data = response.json()

        if 'error' in data:
            print(f"Error: {data['message']}")
            break
        else:
            artists_batch = [artist['name'] for artist in data['topartists']['artist']]
            all_artists.extend(artists_batch)

            if data['topartists']['@attr']['page'] == data['topartists']['@attr']['totalPages']:
                break
            if page == max_pages:
                break

            page +=1
            params['page'] = page
        # wait for the next request
        time.sleep(0.25)
    return all_artists
            


['Kinny Zimmer', 'Oliwka Brazil', 'KPSN', 'Pater', 'LWWL', 'sandra feat peja', 'Bolec', 'V/A - Jestem Stad', 'KROOLIK', 'Ckbtz', 'Robak/Klon', 'Hemp Gru, Dj Fresh', 'DawajHajs', 'Metrowy, Rufijok, Skorup, Bu', 'WYP3', 'Sari Ska Band & Metrowy', 'Kuba Knap i Bonny Larmes', 'TrzyBe', 'DONO/TEWU', 'P.A.K.T.', 'Denetos', 'Dj Kostek', 'BRK i Jahdeck', 'OSWP', 'Kaiteu & Pietz', 'MatiTCU', 'Dsn', 'Al-Fatnujah, Grubson i Junior Stress', 'Arnie', 'RDW, Kaszalot', 'Małolat & Pezet & Ten Typ Mes & Ero', 'PUSHER x OSKA030', 'Mielzky And BennCart', 'Foksi Fejs', 'Plonacy Squot', 'Doniu Feat. 52 Dębiec', 'Rafi/Koni', 'Guli x Homex', 'Stroma', 'Pryzmat Mass!ve', 'Nowe Pokolenie 14/44', 'Sony & PyskatySkurwiel', 'Espoir', 'Łuszy', 'blez', 'Ładzik', 'HIFI Banda feat. Pyskaty, Numer Raz, Kaszalot, Rak, Tede', 'Zielny', 'KALIBER44', 'Wizu', 'QCIEK', 'Supa Skillz Squad', 'dene gutek', 'Pezet\\Noon', 'Skow', 'MALIK & PIERO', 'ordy', 'Wojtas feat. Gutek', 'Flouzuo', 'ROGAL DDL x FAZI', 'Robert-D', 'Fistach'

In [32]:
from collections import Counter
tag = 'polish hip hop'
artists = get_tag_artists(tag)
if artists:
    print(Counter(artists))

Counter({'Kinny Zimmer': 1, 'Oliwka Brazil': 1, 'KPSN': 1, 'Pater': 1, 'LWWL': 1, 'sandra feat peja': 1, 'Bolec': 1, 'V/A - Jestem Stad': 1, 'KROOLIK': 1, 'Ckbtz': 1, 'Robak/Klon': 1, 'Hemp Gru, Dj Fresh': 1, 'DawajHajs': 1, 'Metrowy, Rufijok, Skorup, Bu': 1, 'WYP3': 1, 'Sari Ska Band & Metrowy': 1, 'Kuba Knap i Bonny Larmes': 1, 'TrzyBe': 1, 'DONO/TEWU': 1, 'P.A.K.T.': 1, 'Denetos': 1, 'Dj Kostek': 1, 'BRK i Jahdeck': 1, 'OSWP': 1, 'Kaiteu & Pietz': 1, 'MatiTCU': 1, 'Dsn': 1, 'Al-Fatnujah, Grubson i Junior Stress': 1, 'Arnie': 1, 'RDW, Kaszalot': 1, 'Małolat & Pezet & Ten Typ Mes & Ero': 1, 'PUSHER x OSKA030': 1, 'Mielzky And BennCart': 1, 'Foksi Fejs': 1, 'Plonacy Squot': 1, 'Doniu Feat. 52 Dębiec': 1, 'Rafi/Koni': 1, 'Guli x Homex': 1, 'Stroma': 1, 'Pryzmat Mass!ve': 1, 'Nowe Pokolenie 14/44': 1, 'Sony & PyskatySkurwiel': 1, 'Espoir': 1, 'Łuszy': 1, 'blez': 1, 'Ładzik': 1, 'HIFI Banda feat. Pyskaty, Numer Raz, Kaszalot, Rak, Tede': 1, 'Zielny': 1, 'KALIBER44': 1, 'Wizu': 1, 'QCIEK':

In [36]:
import csv
# this gives comma to after every letter :<
scraped_data_path = 'scraped_data/'
with open(scraped_data_path + 'LastFM_artist.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for artist in artists:
        writer.writerow(artist)


In [51]:
import pickle
with open(scraped_data_path + 'LastFM_artists.pkl', 'wb') as csvfile:
    pickle.dump(artists, csvfile)

In [52]:
with open(scraped_data_path + 'LastFM_artists.pkl', 'rb' ) as f:
    loaded_artist = pickle.load(f)
loaded_artist

['Kinny Zimmer',
 'Oliwka Brazil',
 'KPSN',
 'Pater',
 'LWWL',
 'sandra feat peja',
 'Bolec',
 'V/A - Jestem Stad',
 'KROOLIK',
 'Ckbtz',
 'Robak/Klon',
 'Hemp Gru, Dj Fresh',
 'DawajHajs',
 'Metrowy, Rufijok, Skorup, Bu',
 'WYP3',
 'Sari Ska Band & Metrowy',
 'Kuba Knap i Bonny Larmes',
 'TrzyBe',
 'DONO/TEWU',
 'P.A.K.T.',
 'Denetos',
 'Dj Kostek',
 'BRK i Jahdeck',
 'OSWP',
 'Kaiteu & Pietz',
 'MatiTCU',
 'Dsn',
 'Al-Fatnujah, Grubson i Junior Stress',
 'Arnie',
 'RDW, Kaszalot',
 'Małolat & Pezet & Ten Typ Mes & Ero',
 'PUSHER x OSKA030',
 'Mielzky And BennCart',
 'Foksi Fejs',
 'Plonacy Squot',
 'Doniu Feat. 52 Dębiec',
 'Rafi/Koni',
 'Guli x Homex',
 'Stroma',
 'Pryzmat Mass!ve',
 'Nowe Pokolenie 14/44',
 'Sony & PyskatySkurwiel',
 'Espoir',
 'Łuszy',
 'blez',
 'Ładzik',
 'HIFI Banda feat. Pyskaty, Numer Raz, Kaszalot, Rak, Tede',
 'Zielny',
 'KALIBER44',
 'Wizu',
 'QCIEK',
 'Supa Skillz Squad',
 'dene gutek',
 'Pezet\\Noon',
 'Skow',
 'MALIK & PIERO',
 'ordy',
 'Wojtas feat. Gut

In [46]:
artists

['Kinny Zimmer',
 'Oliwka Brazil',
 'KPSN',
 'Pater',
 'LWWL',
 'sandra feat peja',
 'Bolec',
 'V/A - Jestem Stad',
 'KROOLIK',
 'Ckbtz',
 'Robak/Klon',
 'Hemp Gru, Dj Fresh',
 'DawajHajs',
 'Metrowy, Rufijok, Skorup, Bu',
 'WYP3',
 'Sari Ska Band & Metrowy',
 'Kuba Knap i Bonny Larmes',
 'TrzyBe',
 'DONO/TEWU',
 'P.A.K.T.',
 'Denetos',
 'Dj Kostek',
 'BRK i Jahdeck',
 'OSWP',
 'Kaiteu & Pietz',
 'MatiTCU',
 'Dsn',
 'Al-Fatnujah, Grubson i Junior Stress',
 'Arnie',
 'RDW, Kaszalot',
 'Małolat & Pezet & Ten Typ Mes & Ero',
 'PUSHER x OSKA030',
 'Mielzky And BennCart',
 'Foksi Fejs',
 'Plonacy Squot',
 'Doniu Feat. 52 Dębiec',
 'Rafi/Koni',
 'Guli x Homex',
 'Stroma',
 'Pryzmat Mass!ve',
 'Nowe Pokolenie 14/44',
 'Sony & PyskatySkurwiel',
 'Espoir',
 'Łuszy',
 'blez',
 'Ładzik',
 'HIFI Banda feat. Pyskaty, Numer Raz, Kaszalot, Rak, Tede',
 'Zielny',
 'KALIBER44',
 'Wizu',
 'QCIEK',
 'Supa Skillz Squad',
 'dene gutek',
 'Pezet\\Noon',
 'Skow',
 'MALIK & PIERO',
 'ordy',
 'Wojtas feat. Gut